In [ ]:
import cv2
import os
from pathlib import Path

def split_video_with_overlap(mp4_path: str, num_chunks: int, output_dir: str = "FINAL_cl_24_fps_video_chunks"):
    mp4_path = Path(mp4_path)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)

    # Load video
    cap = cv2.VideoCapture(str(mp4_path))
    assert cap.isOpened(), f"Failed to open video: {mp4_path}"

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Read all frames into memory
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    print(f"Read {len(frames)} frames from video")
    total_frames = len(frames)

    print(f"Loaded {mp4_path.name} with {total_frames} frames at {fps} FPS ({width}x{height})")
    
    # Compute chunk size
    base_chunk_size = total_frames // num_chunks
    extra = total_frames % num_chunks  # to distribute remaining frames if needed

    # Split into chunks with overlap
    start = 0
    for i in range(num_chunks):
        # Compute chunk size with optional frame remainder
        this_chunk_size = base_chunk_size + (1 if i < extra else 0)

        # Add overlap of 1 frame, except for the first chunk
        if i > 0:
            start -= 1
            this_chunk_size += 1

        end = start + this_chunk_size
        print("start (inclusive)=", start, "end (inclusive)=", end - 1, "this_chunk_size=", this_chunk_size)
        chunk_frames = frames[start:end]

        # Output file
        out_path = output_dir / f"{mp4_path.stem}_chunk{i+1}.mp4"
        writer = cv2.VideoWriter(
            str(out_path),
            cv2.VideoWriter_fourcc(*"mp4v"),
            fps,
            (width, height)
        )

        for f in chunk_frames:
            writer.write(f)
        writer.release()
        print(f"Saved chunk {i+1} with {len(chunk_frames)} frames to {out_path}")
        start = end

In [ ]:
import cv2
import os
from pathlib import Path

def split_video_using_timestamps(mp4_path, timestamps, output_dir="cl_start_clips"):
    """
    Splits an MP4 video into clips using OpenCV based on given timestamps.
    Each adjacent clip will have a 1-frame overlap.

    Args:
        mp4_path (str): Path to the input video file.
        timestamps (List[float]): List of timestamps in seconds.
        output_dir (str): Directory where output clips will be saved.

    Returns:
        List[str]: Paths to the saved clips.
    """
    cap = cv2.VideoCapture(mp4_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")

    Path(output_dir).mkdir(parents=True, exist_ok=True)
    output_paths = []

    # Convert timestamps to frame indices
    frame_timestamps = [int(round(ts * fps)) for ts in timestamps]

    for i in range(len(frame_timestamps) - 1):
        start_f = frame_timestamps[i]
        end_f = frame_timestamps[i + 1]

        if i > 0:
            start_f -= 1  # Add 1-frame overlap with previous clip

        cap.set(cv2.CAP_PROP_POS_FRAMES, start_f)
        out_path = os.path.join(output_dir, f"clip_{i:03d}.mp4")
        out = cv2.VideoWriter(out_path, fourcc, fps, (width, height))

        for f in range(start_f, end_f):
            ret, frame = cap.read()
            if not ret:
                break
            out.write(frame)

        out.release()
        output_paths.append(out_path)

    cap.release()
    return output_paths

In [ ]:
timestamps = [35, 40.5]
split_video_using_timestamps("cl.mp4", timestamps)

In [ ]:
timestamps = [35, 40.5]
split_video_using_timestamps("cl.mp4", timestamps)

In [ ]:
timestamps = [0, 7, 13, 18]
split_video_using_timestamps("cl_top_half_24_fps.mp4", timestamps)

In [ ]:
timestamps = [47, 55, 62, 67, 71, 75,81, 84]
split_video_using_timestamps("cl_top_half_24_fps.mp4", timestamps)

In [ ]:
timestamps = [35, 40, 43, 47]
split_video_using_timestamps("cl_top_half_24_fps.mp4", timestamps)

In [ ]:
timestamps = [18, 24, 29, 35]
split_video_using_timestamps("cl_top_half_24_fps.mp4", timestamps)

In [ ]:
split_video_with_overlap("cl_top_half_24_fps.mp4", 16)

In [ ]:
#!/usr/bin/env python3
"""
crop_top_percent.py

Usage:
    python crop_top_percent.py input.mp4 output.mp4 --percent 20

Crops each frame of input.mp4 to its top N% and writes it to output.mp4.
"""

import argparse
import cv2
import sys

def crop_top_percent(input_path: str, output_path: str, percent: float) -> None:
    # Open the source video
    cap = cv2.VideoCapture(input_path)
    if not cap.isOpened():
        print(f"Error: could not open '{input_path}'", file=sys.stderr)
        sys.exit(1)

    # Get source properties
    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    # Compute crop height
    if not (0 < percent <= 100):
        print("Error: --percent must be >0 and ≤100", file=sys.stderr)
        cap.release()
        sys.exit(1)
    crop_h = int(height * (percent / 100.0))
    new_size = (width, crop_h)

    # Prepare the writer at new size
    out = cv2.VideoWriter(output_path, fourcc, fps, new_size)
    if not out.isOpened():
        print(f"Error: could not create '{output_path}'", file=sys.stderr)
        cap.release()
        sys.exit(1)

    # Process frame by frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Crop top percent: rows 0 .. crop_h, all columns
        top_slice = frame[0:crop_h, 0:width]
        out.write(top_slice)

    # Clean up
    cap.release()
    out.release()
    print(f"✅ Cropped top {percent:.1f}% saved to '{output_path}' "
          f"({width}×{crop_h} @ {fps:.2f} fps)")


In [ ]:
crop_top_percent("/Users/usernm/Downloads/cl.mp4", "/Users/username/Downloads/cl_top_30.mp4", 30)
crop_top_percent("/Users/username/Downloads/cl_top_half_6_fps.mp4", "/Users/username/Downloads/cl_top_half_6_fps.mp4", 50)

In [ ]:
# Notebook cell: reduce a video's FPS to 6 and save to a new file
import subprocess
from pathlib import Path

def reduce_fps(input_path: str, output_path: str = None, fps: int = 6) -> Path:
    """
    Reduces the frame rate of an input video to `fps` and writes out a new file.
    
    Args:
        input_path: Path to the source .mp4 file.
        output_path: Optional path for the converted file. 
                     If None, appends '_{fps}fps' to the input filename.
        fps:        Target frames per second (default: 6).
    
    Returns:
        The pathlib.Path of the written output file.
    """
    input_path = Path(input_path)
    if output_path is None:
        output_path = input_path.with_name(f"{input_path.stem}_{fps}fps{input_path.suffix}")
    else:
        output_path = Path(output_path)
    
    cmd = [
        "ffmpeg", "-y",            # overwrite without asking
        "-i", str(input_path),     # input file
        "-r", str(fps),            # set output FPS
        str(output_path)           # output file
    ]
    subprocess.run(cmd, check=True)
    return output_path

# Example usage:
input_mp4  = "/Users/username/Downloads/cl_top_half.mp4"         # ← replace with your file
output_mp4 = "/Users/username/Downloads/cl_top_half_6_fps.mp4"
output_mp4 = reduce_fps(input_mp4, output_mp4, fps=6)
print(f"✅ Saved reduced-FPS video to: {output_mp4}")